In [1]:
import random
import math
import numpy as np
import timeit

# Problem 1 : Loss minimization vs. misclassification

In [2]:
def gradientDescent(x,y,w):
    tmp = (math.e)**(-x*y*w)
    tmp1 = 1/(1+tmp)
    return tmp*tmp1*(-y)*x

In [8]:
def findw(w,dataPoints):
    for i in range(100):
        g = 0
        for j in dataPoints:
            g = g + gradientDescent(j[0],j[1],w)
        g = g/100
        w = w - (0.1)*g
    return w

(c) Consider the case d = 1 and implement gradient descent for minimizing L(w). First consider a
simple “well separable” case with 100 data points (and labels) as follows:
(−50,−1), (−49,−1), ..., (−1,−1), (1,1), (2,1), ..., (50,1).
Initialize w = −1 and run 100 iterations of gradient descent.

In [9]:
dataPoints = []
w = -1
for i in range(1,51):
    dataPoints.append((-i,-1))
    dataPoints.append((i,1))

print(findw(w,dataPoints))

1.6056835787730888


(d) Now suppose we “corrupt” some labels. Specifically, take the 10 points with the highest abso-
lute value of x (i.e., 50, −50, 49, −49, ..., 46, −46) and reverse the sign of their labels. Now
show the result of performing gradient descent, and interpret your result. (Note that we only
corrupted 10% of the labels.)

In [10]:
dataPoints = []
w = -1
for i in range(1,51):
    if i<46:
        dataPoints.append((-i,-1))
        dataPoints.append((i,1))   
    else:
        dataPoints.append((i,-1))
        dataPoints.append((-i,1))
        
answer = findw(w,dataPoints)
print(answer)

1.8461383595192684


# Least squares

In [23]:
def randomSet(m,n):
    arr = [[0 for i in range(n)] for j in range(m)]
    for i in range(m):
        for j in range(n):
            num = random.uniform(-1.0,1.0)
            arr[i][j] = num
    return arr
A = randomSet(1000,500)

(c) Create a ‘random’ dataset for this problem as follows. Set n = 500, m = 2n, and let A have
entries that are random in the interval [−1,1]. Now choose some “hidden” vector x∗ (entries
again random in [−1,1]), and define b as Ax∗ + η, where η is a vector whose coordinates are
Gaussian with mean 0 and variance 0.5.

In [24]:
def randomx():
    arr = [[0 for i in range(1)] for j in range(500)]
    for i in range(500):
        for j in range(1):
            num = random.uniform(-1.0,1.0)
            arr[i][j] = num
    return arr
x = randomx()
n = np.random.normal(loc=0.0, scale=math.sqrt(0.5), size=1000)
AT = np.transpose(A)

Finding B

In [25]:
def find_b():
    b = [0 for i in range(len(n))]
    for i in range(len(A)):
        tmp = 0
        for j in range(len(A[i])):
            tmp += (A[i][j]*x[j][0])
        b[i] = tmp + n[i]
    return b
b = find_b()

Finding Graduent Descent

In [28]:
def gradient(inp):
    return 2*np.subtract(np.dot(np.dot(AT,A),inp),np.dot(AT,b))

def gradient_descent(inp,step_size,n):
    mat = inp
    for _ in range(n):
        mat +=  -step_size * gradient(mat)
    return mat

(d) Run gradient descent with a constant stepsize (say 1/10) starting with x0 = 0, and report the
function value and the distance to the ‘hidden’ x∗ after 50 steps.

In [29]:
start = timeit.default_timer()
init = [0 for i in range(len(x))]
ans = gradient_descent(init,0.1,50)
stop = timeit.default_timer()
print("Time: "+str(stop - start))

tm = n[:]
for i in range(len(A)):
    tmp = 0
    for j in range(len(A[i])):
        tmp += (A[i][j]*ans[j])
    tm[i] = tmp + b[i]
print("Value: " + str(np.linalg.norm(tm)**2))

dist = np.linalg.norm(x-ans)
print("Distance from hidden x " + str(dist))

Time: 2.2058558420000054
Value: 3.1256006167962924e+230
Distance from hidden x 1.2887328832751707e+115


(e) It is well known that least squares regression has a closed form, given by x∗ = (AT A)−1AT b.
Using a numerical library for the inverse, compute x∗ using this formula. Compare the running
time of this method to that of gradient descent in part (d).

In [30]:
def closed_form():
    tmp = np.dot(np.linalg.inv(np.dot(AT,A)),AT)
    ans =  np.dot(tmp,b) 
    return ans

start = timeit.default_timer()
closed= closed_form()
stop = timeit.default_timer()
print("Time: "+str(stop - start))
print("Value:" + str(np.linalg.norm(closed)**2))
print("Distance from hidden x*" + str(np.linalg.norm(x-closed)))

Time: 0.06084523999999192
Value:161.2006930256866
Distance from hidden x*402.17505529999346


# Problem 4 : Convergence of SGD

(c) Implement the SGD algorithm for this f. For concreteness, start with x0 = y0 = 1. Observe the
values of yt for 200 steps with a constant learning rate (say 0.1) and comment on your results.
What happens if you change the learning rate to 0.1/(t + 1) and 0.1/√t + 1? (Note that these
are time-dependent learning rates)

In [31]:
def learningRate1(x,y):
    for t in range(1,201):
        grad_a = 0
        grad_b = 0
        for i in range(1,101):
            if i<=50:
                a = i/100
                b = -1
            else:
                a = (i-100)/100
                b = 1
            grad_a += (2*(x-a))
            grad_b += (2*(y-b))
        x = x - (0.1)*(grad_a/200)
        y = y - (0.1)*(grad_b/200)
        #print("step "+str(i)+" :"+str(y))
    print("For 0.1/: x = " + str(x) +" and "+ " y = " + str(y) )
    return (x,y)

In [32]:
def learningRate2(x,y):
    for t in range(1,201):
        A = 0
        B = 0
        for i in range(1,101):
            if i<=50:
                a = i/100
                b = -1
            else:
                a = (i-100)/100
                b = 1
            A = A + (2*(x-a))
            B = B + (2*(y-b))
        x = x - (0.1/t+1)*(A/200)
        y = y - (0.1/t+1)*(B/200)
    print('For 0.1/(t+1) : x = '+ str(x) +" and "+ " y = " + str(y) )

In [33]:
import math
def learningRate3(x,y):
    for t in range(1,201):
        A = 0
        B = 0
        for i in range(1,101):
            if i<=50:
                a = i/100
                b = -1
            else:
                a = (i-100)/100
                b = 1
            A = A + (2*(x-a))
            B = B + (2*(y-b))
        x = x - (0.1/(math.sqrt(t+1))*(A/200))
        y = y - (0.1/(math.sqrt(t+1))*(B/200))
    print('For 0.1/√(t+1) : x = '+ str(x) +" and "+ " y = " + str(y) )

In [34]:
learningRate1(1,1)
learningRate2(1,1)
learningRate3(1,1)

For 0.1/: x = 0.005000000701980365 and  y = 7.05507775727842e-10
For 0.1/(t+1) : x = 0.004999999999999982 and  y = 1.4697461203939076e-17
For 0.1/√(t+1) : x = 0.07759102595798106 and  y = 0.07295580498289558
